In [1]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    '/Deployment/splitdataset48x48/train',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    '/Deployment/splitdataset48x48/val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 8465 images belonging to 31 classes.
Found 2144 images belonging to 31 classes.


In [3]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'Is', 'J', 'K', 'L', 'M', 'N', 'Name', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'What', 'X', 'Y', 'Your', 'Z', 'blank']


In [4]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(31, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [6]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [7]:
import shutil
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/100


66/66 [==============================] - 208s 3s/step - loss: 3.4226 - accuracy: 0.0319 - val_loss: 3.4137 - val_accuracy: 0.0352
Epoch 2/100
66/66 [==============================] - 125s 2s/step - loss: 3.3912 - accuracy: 0.0392 - val_loss: 3.3134 - val_accuracy: 0.0522
Epoch 3/100
66/66 [==============================] - 99s 1s/step - loss: 3.1969 - accuracy: 0.0692 - val_loss: 2.8451 - val_accuracy: 0.1147
Epoch 4/100
66/66 [==============================] - 115s 2s/step - loss: 2.6415 - accuracy: 0.1516 - val_loss: 2.0094 - val_accuracy: 0.3076
Epoch 5/100
66/66 [==============================] - 131s 2s/step - loss: 1.9827 - accuracy: 0.2899 - val_loss: 1.3344 - val_accuracy: 0.4902
Epoch 6/100
66/66 [==============================] - 102s 2s/step - loss: 1.4404 - accuracy: 0.4285 - val_loss: 0.7707 - val_accuracy: 0.7300
Epoch 7/100
66/66 [==============================] - 102s 2s/step - loss: 1.0035 - accuracy: 0.5971 - val_loss: 0.4144 - val_accuracy: 0.8647
Epoch

KeyboardInterrupt: 

In [9]:
%load_ext tensorboard
%tensorboard --logdir Logs

Launching TensorBoard...

In [10]:
model_json = model.to_json()
with open("H:\Deployment\model\signlanguagedetectionmodel48x48.json",'w') as json_file:
    json_file.write(model_json)
model.save("H:\Deployment\model\signlanguagedetectionmodel48x48.h5")

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
import pickle

# Train your model as shown in your code

# Save the entire model
model.save("my_model.h5")

# Load the saved model
loaded_model = load_model("my_model.h5")

# Convert the model to a byte stream
model_bytes = pickle.dumps(loaded_model)

# Save the byte stream to a .pkl file
with open("my_model.pkl", "wb") as f:
    f.write(model_bytes)
